In [ ]:
from nanome.service import PluginService
from nanome.interface import PluginInstanceRedisInterface

# Set up redis credentials
redis_host = 'redis'
redis_port = 6379

# When your PluginService is running, you can get your channel value from the Logs, or from the query parameter in your open browser.
# Update this value to match that, so that your commands will run against your live workspace.
redis_channel = 'a5100bcd-3b33-48dc-a8b0-67f88182f24e'

plugin_instance = PluginInstanceRedisInterface(redis_host, redis_port, redis_channel)
print('Hello')

In [ ]:
comps = plugin_instance.request_complex_list()
comp = comps[0]
print(comp)


In [ ]:
pip install py2neo

In [ ]:
# neo4j setup
from py2neo import Graph, Node, Relationship

neo4j_uri = 'bolt://neo4j:7687/'
username = 'neo4j'
password = 'F8dwclytew'
graph = Graph(neo4j_uri, auth=(username, password))


In [ ]:
with open("assets/neo4j_query.txt") as f:
    query_string = f.read()
results = graph.run(query_string)
data = results.data()

In [ ]:
from nanome.api.shapes import Shape, Sphere
from nanome.utils import Color
sphere = Sphere()
sphere.radius = 5
sphere.color = Color(255, 0, 0, 100)
sphere = plugin_instance.upload_shapes([sphere])

In [ ]:
query = 'MATCH (n) RETURN n LIMIT 25'
results = graph.run(query)
data = results.data()

In [ ]:
from nanome.api.structure import Atom

atom_list = []
for record in data:
    node = record['n']
    label = node['name']
    print(label)
    atom = Atom()
    atom.labeled = True
    atom.label_text = label
    atom.name = node.__uuid__
    atom_list.append(atom)

In [ ]:
from nanome.api import structure
comp = structure.Complex()
mol = structure.Molecule()
chain = structure.Chain()
res = structure.Residue()

counter = 0
for atom in atom_list:
    print(atom.label_text)
    atom.position.x = counter
    res.add_atom(atom)
    counter += 1

chain.add_residue(res)
mol.add_chain(chain)
comp.add_molecule(mol)

results = plugin_instance.update_structures_deep([comp])